# TODO

* Add actions:
  - Delete/edit feedback option
  - Switch students
* Hook up database
* Make a dashboard layout

* Deploy

### Greying out widgets
When a solution has been graded, navigating to that solution *should display* all the feedback,
but the grader should not be able to alter it (feedback, addfeedbackoption, remark and submit widgets
should be greyed out).

These widgets should also be greyed out when there is another grader already looking at this solution
(otherwise people may waste time grading the same solution, even though we can enforce that the
first grader to submit their feedback "wins" in our business logic).

In [ ]:
from itertools import starmap
from collections import namedtuple
import random
import time

from IPython import display
from ipywidgets import Button, Text, SelectMultiple, HBox, VBox, Textarea, Image, Label
import pandas
import jinja2
from pony import orm

import db

In [ ]:
### Javascript kludges to add autocompletion to text boxes and enable multiple selections

autocomplete = jinja2.Template("""
  $( function() {
    var availableTags = [
    {%- for option in options %}
        "{{ option }}",
    {%- endfor %}
    ];
    $( ":input[{{ selector }}]" ).autocomplete({
      minLength: 3,
      source: availableTags
    });
  } );
""")

prevent_unselect = """
window.onmousedown = function (e) {
    var el = e.target;
    if (el.tagName.toLowerCase() == 'option' && el.parentNode.hasAttribute('multiple')) {
        e.preventDefault();

        // toggle selection
        if (el.hasAttribute('selected')) el.removeAttribute('selected');
        else el.setAttribute('selected', '');

        // hack to correct buggy behavior
        var select = el.parentNode.cloneNode(true);
        el.parentNode.parentNode.replaceChild(select, el.parentNode);
    }
}
"""

In [ ]:
### Views -- all the individual widgets and UI state

## Immutable app state
with orm.db_session:
    n_submissions = db.Submission.select().count()
    n_problems = db.Problem.select().count()
    students = tuple(starmap('{} ({} {})'.format, orm.select((s.id, s.first_name, s.last_name) for s in db.Student)))
    problems = tuple(orm.select(p.name for p in db.Problem))

## Mutable app state

# the solution that we are pointing to
submission_id = 1
problem_id = 1

## Solution selection (submission x problem) matrix
prev_stud = Button(icon='fa-fast-backward', description=' previous')
submit_and_next_stud = Button(icon='fa-fast-forward', description=' submit & next', button_style='warning')
next_stud = Button(icon='fa-forward', description=' next')
jump_to_stud = Text(description='Jump to', placeholder='student # or name')
submission_label = Label(description='current submission:')

prev_problem = Button(icon='fa-arrow-circle-up', description=' previous problem')
submit_and_next_problem = Button(icon='fa-arrow-circle-down', description=' submit & next problem',
                                 button_style='warning')
next_problem = Button(icon='fa-arrow-circle-down', description=' next problem')
jump_to_problem = Text(description='Jump to', placeholder='problem #')
problem_label = Label(description='current submission:')

## Add feedback option
new_option = Text(placeholder='New feedback option')

## Solution and FeedbackOption
options_feedback = SelectMultiple(name='feedback')
student_specific = Textarea(placeholder='Student-specific feedback')
answer = Image(format='png', width='100%')

In [ ]:
### Controllers  -- hooking views (widgets) up to models

## Utilities

def add_option(target, value):
    with target.hold_trait_notifications():
        target.options = target.options + [value]
    
def load_image(path):
    with open(path, 'rb') as f:
        return f.read()

# render the solution widgets
def render_solution():
    with orm.db_session:
        # this works because Solution has '(submission, problem)' as its PrimaryKey
        solution = db.Solution.get(submission=submission_id, problem=problem_id)
        # get actual data back before we finish the database transaction
        possible_feedback = [f.text for f in solution.problem.feedback_options]
        selected_feedback = [f.text for f in solution.feedback]

    # update the UI state
    submission_label.value = str(submission_id)
    problem_label.value = str(problem_id)
    answer.value= load_image(solution.image_path)
    options_feedback.options = possible_feedback
    options_feedback.value = selected_feedback
    student_specific.value = solution.remarks
    
def render_after(callback):
    
    def _(sender):
        callback(sender)
        render_solution()

    return _


## actual controllers - abusing decorator syntax FTW!

@prev_stud.on_click
@render_after
def _(sender):
    global submission_id
    if submission_id > 1:
        submission_id -= 1
    # else flash a message "already on first student"

@next_stud.on_click
@render_after
def _(sender):
    global submission_id
    if submission_id < n_submissions:
        submission_id += 1
    # else flash a message "already on last student"

@submit_and_next_stud.on_click 
@render_after
def _(sender):
    global submission_id
    if submission_id < n_submissions:
        submission_id += 1
        # TODO: update db
    # else flash a message "already on first student"

@prev_problem.on_click
@render_after
def _(sender):
    global problem_id
    if problem_id > 1:
        problem_id -=1
    # else flash a message "already on first problem"
    
@next_problem.on_click
@render_after
def _(sender):
    global problem_id
    if problem_id < n_problems:
        problem_id +=1
    # else flash a message "already on last problem"
    
@submit_and_next_problem.on_click
@render_after
def _(sender):
    global problem_id
    if problem_id < n_problems:
        problem_id +=1
        # TODO: update db
    # else flash a message "already on last problem"

@new_option.on_submit
def _(sender):
    # add to database
    with orm.db_session:
        if sender.value.strip() and not db.FeedbackOption.exists(text=sender.value):
            db.FeedbackOption(text=sender.value, problems=db.Problem[problem_id])
            # update UI state manually rather than doing a full re-render
            add_option(options_feedback, sender.value)
    sender.value = ''

In [ ]:
### Widget layout and rendering

widgets = VBox([
        HBox([prev_stud, next_stud, submit_and_next_stud, jump_to_stud, submission_label]),
        HBox([next_problem, prev_problem, submit_and_next_problem, jump_to_problem, problem_label]),
        HBox([VBox([options_feedback, new_option]), student_specific]),
        answer,
])

# display widgets
render_solution()
display.display(widgets)

# apply javascript kludges
for widget, options in [(jump_to_stud, students), (jump_to_problem, problems)]:
    selector = "placeholder='{}'".format(widget.placeholder)
    display.display_javascript(display.Javascript(autocomplete.render(selector=selector, options=options)))
display.display_javascript(prevent_unselect)